In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests
import random
import math
import pathlib 

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait, get_paper_data, mod_names, check_google, similar_names, search_names
from modules.data_manipulation_functions import remove_commas, check_files, concat_function


In [2]:
# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)
g_driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")

In [18]:
# Load the datasets from the working directory
# The datasets in the working directory have already 
# been cleaned.
control = pd.read_excel(work_path / "control.xlsx")
lateral = pd.read_excel(work_path / "lateral.xlsx")

# Load the name modification dataset
name_mod_lateral = pd.read_excel(input_path / "name_mod_lateral.xlsx")

# Data type: This is the type of data that we are using.
# The control group members only have one university url because they
# did not move. The lateral group members have two urls.
data_type = "lateral"

#This step sets the data 
data = lateral
name_mod = name_mod_lateral

# MERGE: Right join the name mod data to get the observations with a different last name
alt_name_diff_last_name = pd.merge(data, name_mod[["ID", "diff_last_name"]], how = "right", left_on = ["ID"], right_on = ["ID"])

# Subset to the non-missing different last names
alt_name_diff_last_name = alt_name_diff_last_name[alt_name_diff_last_name["diff_last_name"].notnull()]
# Create a copy of the alternate last name observations with a different last name
alt_name_diff_last_name["LastName"] = alt_name_diff_last_name["diff_last_name"]
# Drop the diff_last_name column
alt_name_diff_last_name = alt_name_diff_last_name.drop(["diff_last_name"], axis = 1)

# Add the observations onto the main dataset
data = pd.concat([data, alt_name_diff_last_name], ignore_index = True)

# Flag any observations that now show up multiple times
data["multi_obs"] = data["ID"].duplicated(keep=False)

data

,Unnamed: 0,Short URL Destination,Short URL Origin,FirstName,LastName,ID,Lateral,LateralYear,Origin School,Destination School,multi_obs
0,0,duke.edu,upenn.edu,Matthew,Adler,1,1,2012,University of Pennsylvania Law School,Duke University School of Law,False
1,1,duke.edu,northwestern.edu,Michael,Frakes,80,1,2016,Northwestern University Pritzker School of Law,Duke University School of Law,False
2,2,duke.edu,virginia.edu,Brandon,Garrett,88,1,2018,University of Virginia School of Law,Duke University School of Law,False
3,3,duke.edu,gwu.edu,H.,Powell,216,1,2012,George Washington University Law School,Duke University School of Law,False
4,4,duke.edu,uc.edu,Darrell,Miller,182,1,2013,University of Cincinnati College of Law,Duke University School of Law,False
...,...,...,...,...,...,...,...,...,...,...,...
284,284,udayton.edu,fcsl.edu,Christopher,Roederer,230,1,2018,Florida Coastal School of Law,University of Dayton School of Law,False
285,285,umkc.edu,valpo.edu,Del,Wright,281,1,2017,Valparaiso University Law School,University of Missouri-Kansas City School of Law,False
286,286,umt.edu,unl.edu,Sandi,Zellmer,287,1,2018,University of Nebraska College of Law,University of Montana Alexander Blewett III Sc...,False
287,194,sandiego.edu,colorado.edu,Miranda,Perry,76,1,2013,University of Colorado Law School,University of San Diego School of Law,True


In [19]:
# Create a list to store the alternate name data
alt_names_data = []
# Create the alt-names dataset column list
alt_names_columns = ["ID", "FirstName", "LastName", "fm_names", "err_fm_names", "multi_obs"]

# Check to see if the file for the alternate names data already exists.
# If it does, we only want to looks for the missing observations
alt_names_file = intr_path / "alt_names.xlsx"
if alt_names_file.exists():
    print("Data already exists. Names that have already been scraped will be skipped")
    # Set the append flag to 1
    append = 1
    # Create the dataset of existing alt names.
    df_existing_alt_names = pd.read_excel(alt_names_file)
    # Complete a left outer join of the existing alt names and the lateral/control data to get 
    # a list of the names that we still need to scrape alt names for.
    data = pd.merge(data, df_existing_alt_names["ID"], how = "outer", left_on = "ID", right_on = "ID", indicator=True)
    data = data[data['_merge'] == 'left_only']
    data = data.drop(["_merge"], axis = 1)
else:
    # Set the append flag to zero because we won't have any data to append
    append = 0
# Data contains the observations that were not available in the existing alt_names file.
# If there is no existing file, it just contains the lateral/control data.
data.reset_index(inplace = True)

Data already exists. Names that have already been scraped will be skipped


In [20]:
data

,index,Unnamed: 0,Short URL Destination,Short URL Origin,FirstName,LastName,ID,Lateral,LateralYear,Origin School,Destination School,multi_obs
0,90,90,berkeley.edu,osu.edu,Steven,Davidoff,57,1,2013,The Ohio State University Michael E. Moritz Co...,UC Berkeley Law,True
1,91,90,berkeley.edu,osu.edu,Steven,Solomon,57,1,2013,The Ohio State University Michael E. Moritz Co...,UC Berkeley Law,True
2,195,194,sandiego.edu,colorado.edu,Miranda,Fleischer,76,1,2013,University of Colorado Law School,University of San Diego School of Law,True
3,196,194,sandiego.edu,colorado.edu,Miranda,Perry,76,1,2013,University of Colorado Law School,University of San Diego School of Law,True


In [21]:

#This loop goes through each name
for i in range(len(data)):
    #This section gets the professor's information from the dataframe 
    # Get variable values from the dataframe
    prof_id = data['ID'][i]
    mid_first_name = data['FirstName'][i]
    last_name = data['LastName'][i]
    full_name = mid_first_name + ' ' +  last_name
    #This line gets the school URLs from the dataframe
    if data_type == "lateral":
        school_url = [data['Short URL Origin'][i], data['Short URL Destination'][i]]
        school = data['Origin School'][i]
        new_school = data['Destination School'][i]
    elif data_type == "control":
        school_url = [data['Short URL Origin'][i]]
        school = data['Origin School'][i]
    multi_obs = data["multi_obs"][i]

    # Print the name that we are considering
    print(full_name)
    
    # Search by author to find potential alternative first and middle names:
    print(school_url)
    fm_names, err_fm_names = search_names(mid_first_name, last_name, school_url, driver, g_driver)


    # Create a list of values to append to the dataframe
    # We convert fm_names and err_fm_names to lists of strings during this step
    values_alt_names = [prof_id, mid_first_name, last_name, fm_names, err_fm_names, multi_obs]
    dict_values_alt_names = dict(zip(alt_names_columns, values_alt_names))
    alt_names_data.append(dict_values_alt_names)

Steven Davidoff
['osu.edu', 'berkeley.edu']
URL: osu.edu
osu.edu in: https://moritzlaw.osu.edu/briefing-room/tag/steven-m-davidoff/
URL: osu.edu
osu.edu in: https://moritzlaw.osu.edu/briefing-room/tag/steven-m-davidoff/
Steven Solomon
['osu.edu', 'berkeley.edu']
URL: osu.edu
osu.edu in: https://osupublicationarchives.osu.edu/?a=d&d=LTN19681121-01.2.28
URL: osu.edu
URL: berkeley.edu
berkeley.edu in: https://www.law.berkeley.edu/our-faculty/faculty-profiles/steven-davidoff-solomon/
URL: osu.edu
URL: berkeley.edu
Miranda Fleischer
['colorado.edu', 'sandiego.edu']
URL: colorado.edu
colorado.edu in: https://lawweb.colorado.edu/files/vitae/fleischerm.pdf
Miranda Perry
['colorado.edu', 'sandiego.edu']
URL: colorado.edu
colorado.edu in: https://lawweb.colorado.edu/files/vitae/fleischerm.pdf


In [22]:
# Add the names back into the dataframe of alternate names
df_alt_names = pd.DataFrame(columns = alt_names_columns)

# Only complete these steps if we have data to add
if alt_names_data:
    # Append all of the data to the ouput dataset and output to Excel
    df_alt_names = df_alt_names.append(alt_names_data)

    # MERGE: Merge on the other variables from the lateral data
    alt_name_full = pd.merge(df_alt_names, lateral.drop(["FirstName", "LastName"], axis = 1), how = "left", left_on = "ID", right_on = "ID")

else:
    alt_name_full = df_alt_names_final

# # If there was already output data, we append the data to the existing data
if append == 1:
    alt_names = pd.concat([df_existing_alt_names, alt_name_full], ignore_index = True)
else:
    alt_names = alt_name_full

# DROP: Drop extra columns
# alt_names.drop(["diff_last_name", "mid_first_name", "last_name"], axis = 1, inplace = True)

# Export to Excel
alt_names.to_excel(intr_path / "alt_names.xlsx")

In [23]:
alt_names

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,ID,Short URL Destination,Short URL Origin,FirstName,LastName,Lateral,LateralYear,Origin School,Destination School,fm_names,err_fm_names,multi_obs
0,0,0.0,0.0,0.0,0.0,1,duke.edu,upenn.edu,Matthew,Adler,1,2012,University of Pennsylvania Law School,Duke University School of Law,"Matthew, Matthew D.",Matthew H.,False
1,1,1.0,1.0,1.0,1.0,2,gsu.edu,avemarialaw.edu,Edward,Afield,1,2016,Ave Maria School of Law,Georgia State College of Law,W. Edward,NaN,False
2,2,2.0,2.0,2.0,2.0,3,utexas.edu,bc.edu,Richard,Albert,1,2017,Boston College Law School,University of Texas School of Law,Richard,NaN,False
3,3,3.0,3.0,3.0,3.0,4,tamu.edu,wisc.edu,Lisa,Alexander,1,2016,University of Wisconsin Law School,Texas A&M University School of Law,Lisa T.,Lisa E.,False
4,4,4.0,4.0,4.0,4.0,5,american.edu,suffolk.edu,Hilary,Allen,1,2018,Suffolk University Law School,American University Washington College of Law,Hilary J.,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,284,284.0,284.0,284.0,284.0,97,cornell.edu,umaryland.edu,James,Grimmelmann,1,2016,University of Maryland Francis King Carey Scho...,Cornell Law School,James,NaN,False
285,90,NaN,NaN,NaN,NaN,57,berkeley.edu,osu.edu,Steven,Davidoff,1,2013,The Ohio State University Michael E. Moritz Co...,UC Berkeley Law,"Steven, Steven M.",,True
286,90,NaN,NaN,NaN,NaN,57,berkeley.edu,osu.edu,Steven,Solomon,1,2013,The Ohio State University Michael E. Moritz Co...,UC Berkeley Law,"Steven, Steven Davidoff",Steven A.,True
287,194,NaN,NaN,NaN,NaN,76,sandiego.edu,colorado.edu,Miranda,Fleischer,1,2013,University of Colorado Law School,University of San Diego School of Law,Miranda Perry,,True
